In [1]:
%matplotlib widget
import huffTreeUtilities as hf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy
from CompStats import getCompressionStats
import copy

In [2]:
I = plt.imread('image.jpg', 'uint8')
Ic = I.copy()
Ic_red = hf.loadHuffableImage(Ic[..., 0]) # it is 2-D array, it removes the color channel already
red_chan = Ic[...,0]
encoder_red, decoder_red = hf.buildHuffPair(red_chan)
en_red = ''.join(encoder_red[pix] for pix in red_chan.ravel()) # string of encoded image

Ic_green = hf.loadHuffableImage(Ic[..., 1]) # it is 2-D array, it removes the color channel already
green_chan = Ic[...,1]
encoder_green, decoder_green = hf.buildHuffPair(green_chan)
en_green = ''.join(encoder_green[pix] for pix in green_chan.ravel()) # string of encoded image

Ic_blue = hf.loadHuffableImage(Ic[..., 2]) # it is 2-D array, it removes the color channel already
blue_chan = Ic[...,2]
encoder_blue, decoder_blue = hf.buildHuffPair(blue_chan)
en_blue = ''.join(encoder_blue[pix] for pix in blue_chan.ravel()) # string of encoded image
# plt.close('all')
# print(en_red)
# encoder_red

In [3]:
print(len(en_red))
print(len(Ic[...,0].ravel())*8)

1044181
2018400


In [4]:
def find_min_length(arr):
    # ['101010','100','11111']
    min_str = arr[0]
    for i in arr:
        if len(i) < len(min_str):
            min_str = i
    return min_str

In [5]:
def substitute(encoder, nearby):
    sub = {}
    intensity_list = np.arange(0, 256)
    for i in intensity_list:
        if i < 256 - nearby:
            arr = [encoder[x] for x in range(i, i+nearby)]
            sub[i] = find_min_length(arr)
        else:
            arr = [encoder[x] for x in range(256 - nearby, 256)]
            sub[i] = find_min_length(arr)
    return sub

In [6]:
def sub_bin(encoder, nearby):
    sub = copy.deepcopy(encoder)
    intensity_list = np.arange(0,256)
    for i in range(len(intensity_list)//nearby):
        r = range(i*nearby, (i+1)*nearby)
        min_str = find_min_length([encoder[x] for x in r])
        for j in r:
            sub[j] = min_str
    return sub           

In [7]:
cp_red = substitute(encoder_red, 20)
cp_blue = substitute(encoder_blue, 20)
cp_green = substitute(encoder_green, 20)
# for i in range(256):
#     print(i, len(cp_red[i]), len(encoder_red[i]))
encoder_red[50]
cp_red[50]

'001100011010'

In [8]:
en_cp_red = ''.join(cp_red[pix] for pix in red_chan.ravel()) # string of encoded image
en_cp_green = ''.join(cp_green[pix] for pix in green_chan.ravel()) # string of encoded image
en_cp_blue = ''.join(cp_blue[pix] for pix in blue_chan.ravel()) # string of encoded image
# decoder_cp_red = dict((cp_red[key], key) for key in cp_red)
# decoder_cp_blue = dict((cp_blue[key], key) for key in cp_blue)
# decoder_cp_green = dict((cp_green[key], key) for key in cp_green)
print(len(en_cp_red)) # my stuff
print(len(en_red)) # huffman

1036416
1044181


In [9]:
cp_red[141]

'0001011100000'

In [10]:
def decodeImage(code, decoder):
    '''
    Decode the code string
    Input: str of encoded code in huffman encoding
    Output: list of intensities that = to I.ravel()
    '''
    decode = np.zeros((Ic_red.ravel()).shape)
    current_index = ''
    index = 0
    for num in code:
        current_index = current_index + num
        if current_index in decoder:
            decode[index] = (decoder[current_index])
            current_index = ''
            index += 1
    return decode

In [11]:
# Form the decoded sequences
# red
re_cp_red = decodeImage(en_cp_red, decoder_red)
# green
re_cp_green = decodeImage(en_cp_green, decoder_green)
# blue
re_cp_blue = decodeImage(en_cp_blue, decoder_blue)

In [12]:
# I_reconstruct = np.reshape(s, red_chan) 
# reconstruct the image
red_reconst = np.reshape(re_cp_red, Ic_red.shape)
green_reconst = np.reshape(re_cp_green, Ic_green.shape)
blue_reconst = np.reshape(re_cp_blue, Ic_blue.shape)
I_reconstructed = np.zeros(I.shape)
I_reconstructed[...,0] = red_reconst
I_reconstructed[...,1] = green_reconst
I_reconstructed[...,2] = blue_reconst
print(I_reconstructed.size)

756900


In [13]:
# Form the decoded sequences
# red
re_red = decodeImage(en_red, decoder_red)
# green
re_green = decodeImage(en_green, decoder_green)
# blue
re_blue = decodeImage(en_blue, decoder_blue)

In [14]:
# I_reconstruct = np.reshape(s, red_chan) 
# reconstruct the image
red_reconst_x = np.reshape(re_red, Ic_red.shape)
green_reconst_x = np.reshape(re_green, Ic_green.shape)
blue_reconst_x = np.reshape(re_blue, Ic_blue.shape)
Ir = np.zeros(I.shape)
Ir[...,0] = red_reconst_x
Ir[...,1] = green_reconst_x
Ir[...,2] = blue_reconst_x
# print(I_reconstructed)

In [15]:
f, axx = plt.subplots(1,3, figsize = (10,3), sharex = True, sharey = True)
axx[0].imshow(I)
axx[0].set_title("original")
axx[1].imshow(Ir.astype('uint8'))
axx[1].set_title("Huffman")

axx[2].imshow(I_reconstructed.astype('uint8'))
axx[2].set_title("5 nearby lossy")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '5 nearby lossy')

In [16]:
plt.figure()
# plt.imshow(I - I_reconstructed)
dif = ((abs(I - I_reconstructed) + 1).astype('uint8'))
# plt.imshow(np.log(dif))
dif = dif/dif.ravel().max()
plt.imshow(dif, cmap = 'gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
#The histogram bins are just let's say 256 bins in [0,1]
bins = np.arange(256)/256
f1, axarr = plt.subplots(1,2, figsize=(10, 3))
Is = I/I.ravel().max()
axarr[0].imshow(I)
axarr[1].hist(Is[...,0].ravel(), bins, alpha = .6, label = 'red', color = 'r')
axarr[1].hist(Is[...,1].ravel(), bins, alpha = .6, label = 'green', color = 'g')
axarr[1].hist(Is[...,2].ravel(), bins, alpha = .6, label = 'blue', color = 'b')
axarr[1].legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
#The histogram bins are just let's say 256 bins in [0,1]
bins = np.arange(256)/256
f1, axarr = plt.subplots(1,2, figsize=(10, 3))
I_reconstructed = I_reconstructed/I_reconstructed.ravel().max()
axarr[0].imshow(I_reconstructed)
axarr[0].set_title('20 lossy')
axarr[1].hist(I_reconstructed[...,0].ravel(), bins, alpha = .6, label = 'red', color = 'r')
axarr[1].hist(I_reconstructed[...,1].ravel(), bins, alpha = .6, label = 'green', color = 'g')
axarr[1].hist(I_reconstructed[...,2].ravel(), bins, alpha = .6, label = 'blue', color = 'b')
axarr[1].legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')